In [12]:
import json
import pandas as pd
from datetime import datetime

In [13]:
file_path = "../../data/raw/facebook_top_all_20250304.json"

with open(file_path, "r", encoding="utf-8") as file:
    json_data = json.load(file)
print(json_data)


[{'title': 'Meta under fire for auto-following Trump & Vance, Blocking Democrat hashtags', 'score': 14510, 'id': '1i6nu9x', 'url': 'https://techissuestoday.com/meta-auto-follow-trump-vance-blocks-democrat-hashtags/', 'author': 'madd74', 'body': '', 'created_utc': '2025-01-21T23:08:56', 'num_comments': 373, 'permalink': '/r/facebook/comments/1i6nu9x/meta_under_fire_for_autofollowing_trump_vance/', 'flair': 'Site Update "error"', 'nsfw': False, 'awards': 0}, {'title': 'Mark Zuckerberg helping to institute a nationwide abortion ban. Gilead inching closer to fruition', 'score': 10551, 'id': '1idw3ep', 'url': 'https://www.theguardian.com/world/2025/jan/29/abortion-pills-instagram-shadow-banning?utm_source=fark&utm_medium=website&utm_content=link&ICID=ref_fark', 'author': 'stfuandgovegan', 'body': '', 'created_utc': '2025-01-31T01:50:28', 'num_comments': 405, 'permalink': '/r/facebook/comments/1idw3ep/mark_zuckerberg_helping_to_institute_a_nationwide/', 'flair': 'News Article', 'nsfw': False

In [14]:
df = pd.DataFrame(json_data)

In [15]:
df.head()

,title,score,id,url,author,body,created_utc,num_comments,permalink,flair,nsfw,awards
0,Meta under fire for auto-following Trump & Van...,14510,1i6nu9x,https://techissuestoday.com/meta-auto-follow-t...,madd74,,2025-01-21T23:08:56,373,/r/facebook/comments/1i6nu9x/meta_under_fire_f...,"Site Update ""error""",False,0
1,Mark Zuckerberg helping to institute a nationw...,10551,1idw3ep,https://www.theguardian.com/world/2025/jan/29/...,stfuandgovegan,,2025-01-31T01:50:28,405,/r/facebook/comments/1idw3ep/mark_zuckerberg_h...,News Article,False,0
2,Meta has agreed to pay $25 million after being...,9835,1id8sg9,https://www.the-express.com/news/politics/1620...,TheExpressUS,,2025-01-30T05:50:30,1143,/r/facebook/comments/1id8sg9/meta_has_agreed_t...,News Article,False,0
3,Meta denies forcing accounts to follow Donald ...,8194,1i864c8,https://www.techspot.com/news/106464-meta-deni...,madd74,,2025-01-23T21:23:41,180,/r/facebook/comments/1i864c8/meta_denies_forci...,News Article,False,0
4,Laid-off Meta employees blast Zuckerberg in fo...,5645,1iphear,https://fortune.com/2025/02/13/laid-off-meta-e...,Healthy_Block3036,,2025-02-15T00:17:05,156,/r/facebook/comments/1iphear/laidoff_meta_empl...,News Article,False,0


In [16]:
# Feature engineering
df['created_utc'] = pd.to_datetime(df['created_utc'])
df['created_hour'] = df['created_utc'].dt.hour
df['created_dayofweek'] = df['created_utc'].dt.dayofweek

In [17]:
# Combine text fields (title + body) for NLP models
df['combined_text'] = df['title'].fillna('') + ' ' + df['body'].fillna('')

In [18]:
# Clean URLs and special chars from text
df['cleaned_text'] = df['combined_text'].str.replace(r'http\S+', '', regex=True).str.replace(r'\W+', ' ', regex=True)

In [19]:
# Encode categorical data
df['flair_encoded'] = pd.factorize(df['flair'])[0]


In [20]:
# Drop irrelevant columns
df = df.drop(columns=['url', 'id', 'permalink', 'created_utc', 'author'])

In [21]:
df.head()

,title,score,body,num_comments,flair,nsfw,awards,created_hour,created_dayofweek,combined_text,cleaned_text,flair_encoded
0,Meta under fire for auto-following Trump & Van...,14510,,373,"Site Update ""error""",False,0,23,1,Meta under fire for auto-following Trump & Van...,Meta under fire for auto following Trump Vance...,0
1,Mark Zuckerberg helping to institute a nationw...,10551,,405,News Article,False,0,1,4,Mark Zuckerberg helping to institute a nationw...,Mark Zuckerberg helping to institute a nationw...,1
2,Meta has agreed to pay $25 million after being...,9835,,1143,News Article,False,0,5,3,Meta has agreed to pay $25 million after being...,Meta has agreed to pay 25 million after being ...,1
3,Meta denies forcing accounts to follow Donald ...,8194,,180,News Article,False,0,21,3,Meta denies forcing accounts to follow Donald ...,Meta denies forcing accounts to follow Donald ...,1
4,Laid-off Meta employees blast Zuckerberg in fo...,5645,,156,News Article,False,0,0,5,Laid-off Meta employees blast Zuckerberg in fo...,Laid off Meta employees blast Zuckerberg in fo...,1


In [23]:
import os

# Ensure the directory exists
save_dir = r"../../data/processed"

# Save to CSV
csv_path = os.path.join(save_dir, 'preprocessed_data.csv')
json_path = os.path.join(save_dir, 'preprocessed_data.json')
df.to_csv(csv_path, index=False)
# df.to_json(csv_path.replace('.csv', '.json'), orient='records', lines=True)
df.to_json(json_path, orient='records', indent=4) 
